In [1]:
import os
import numpy as np
import pandas as pd



# 모델 수 및 클라이언트 수
M = 53  # 모델 수 (src_001~src_053)
J = 53  # 클라이언트 수 (tgt_001~tgt_053)

# 경로 설정 (파일들이 저장된 디렉토리)
dir_path = "./raw/"  # 파일이 업로드된 디렉토리 경로

# probs, labels 초기화
probs = {i: {} for i in range(1, M+1)}  # probs[i][j] -> 확률 값
labels = {j: [] for j in range(1, J+1)}  # labels[j] -> 라벨 값

# CSV 파일 로드 및 전처리
for i in range(1, M+1):  # 모델 번호
    for j in range(1, J+1):  # 클라이언트 번호
        file_path = os.path.join(dir_path, f"src_{str(i).zfill(3)}/probs/tgt_{str(j).zfill(3)}.csv")
        
        # 각 파일이 존재하면 읽기
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            
            # 확률(p_A)과 라벨(label_num) 추출
            probs[i][j] = df['p_A'].values  # 모델 i의 클라이언트 j에 대한 p(A) 확률
            labels[j] = df['label_num'].values  # 클라이언트 j에 대한 라벨
            
        else:
            print(f"Warning: File {file_path} not found!")

# 데이터 확인
print(probs[1][1][:5])  # 첫 번째 모델과 첫 번째 클라이언트에 대한 확률 예시
print(labels[1][:5])  # 첫 번째 클라이언트에 대한 라벨 예시

[0.32082132 0.21206881 0.63703084 0.30735803 0.59266663]
[0 0 0 0 1]


In [2]:
print(probs)
print(labels)

{1: {1: array([0.32082132, 0.21206881, 0.63703084, 0.30735803, 0.59266663,
       0.2814056 , 0.76629364, 0.62245935, 0.65135491, 0.7879312 ,
       0.54673815, 0.54673815, 0.51561993, 0.5       , 0.79818678,
       0.74316794, 0.86703575, 0.45326188, 0.71859437, 0.79818678,
       0.40733343, 0.67917871, 0.5774954 , 0.45326188, 0.23370634,
       0.30735803, 0.11920292, 0.71859437, 0.73105854, 0.4378235 ,
       0.20181322, 0.56217653, 0.21206881, 0.46879065, 0.56217653,
       0.59266663, 0.65135491, 0.89330941, 0.256832  , 0.30735803]), 2: array([0.77729988, 0.54673815, 0.83548355, 0.53120941, 0.34864515,
       0.56217653, 0.65135491, 0.46879065, 0.5774954 , 0.66541058,
       0.37754068, 0.53120941, 0.33458945, 0.37754068, 0.73105854,
       0.30735803, 0.2814056 , 0.62245935, 0.51561993, 0.53120941,
       0.4843801 , 0.256832  , 0.42250463, 0.46879065, 0.79818678,
       0.8080672 , 0.63703084, 0.94334757, 0.66541058, 0.256832  ,
       0.51561993, 0.62245935, 0.754915  , 0.8175

In [3]:
import numpy as np

# ---------- 내부 유틸(쌍별 연산의 벡터화) ----------

def _pairwise_soft_mcc_from_s(S, eps=1e-9):
    """
    S: (R, n) with S[r,i] = P(model i correct on item r)
    return: (n, n) similarity in [0,1]
    """
    S = S.astype(np.float64, copy=False)
    one_minus_S = 1.0 - S

    A = S.T @ S
    B = S.T @ one_minus_S
    C = B.T
    D = one_minus_S.T @ one_minus_S

    denom = np.sqrt((A + B) * (A + C) * (B + D) * (C + D)) + eps
    phi = (A * D - B * C) / denom
    phi = np.clip(phi, -1.0, 1.0)

    S_phi = 0.5 * (phi + 1.0)
    np.fill_diagonal(S_phi, 1.0)
    return S_phi

def _pairwise_counts_from_binary(C):
    """
    C: (R, n) with C in {0,1} meaning "model i is correct on item r"
    returns A,B,C2,D and N=R
    """
    C = C.astype(np.float64, copy=False)
    one_minus_C = 1.0 - C
    A = C.T @ C
    B = C.T @ one_minus_C
    C2 = B.T
    D = one_minus_C.T @ one_minus_C
    N = C.shape[0]
    return A, B, C2, D, N

def _pairwise_kappa_from_binary(C, eps=1e-9):
    A, B, C2, D, N = _pairwise_counts_from_binary(C)
    Po = (A + D) / float(N)
    Pe = ((A + B) * (A + C2) + (C2 + D) * (B + D)) / float(N * N)
    kappa = (Po - Pe) / (1.0 - Pe + eps)
    kappa = np.clip(kappa, -1.0, 1.0)
    S = 0.5 * (kappa + 1.0)
    np.fill_diagonal(S, 1.0)
    return S

def _pairwise_hard_mcc_from_binary(C, eps=1e-9):
    A, B, C2, D, _ = _pairwise_counts_from_binary(C)
    denom = np.sqrt((A + B) * (A + C2) * (B + D) * (C2 + D)) + eps
    phi = (A * D - B * C2) / denom
    phi = np.clip(phi, -1.0, 1.0)
    S = 0.5 * (phi + 1.0)
    np.fill_diagonal(S, 1.0)
    return S

def _pairwise_cosine_from_matrix(Z, eps=1e-12):
    """
    Z: (R, n) with Z[r,i] = (2q_r-1)*(2p_i(r)-1)
    returns cosine similarity in [0,1]
    """
    Z = Z.astype(np.float64, copy=False)
    dot = Z.T @ Z
    norms = np.linalg.norm(Z, axis=0, keepdims=True)
    denom = (norms.T @ norms) + eps
    cos = dot / denom
    cos = np.clip(cos, -1.0, 1.0)
    S = 0.5 * (cos + 1.0)
    np.fill_diagonal(S, 1.0)
    return S

def _pairwise_balanced_jaccard_from_binary(C):
    A, B, C2, D, _ = _pairwise_counts_from_binary(C)
    denom_correct = A + B + C2
    denom_error = B + C2 + D
    # zero-division 방지: 분모가 0이면 해당 자카드는 1로 둠(그 slice에선 완전 일치)
    Jc = np.divide(A, denom_correct, out=np.ones_like(A), where=(denom_correct > 0))
    Je = np.divide(D, denom_error, out=np.ones_like(D), where=(denom_error > 0))
    S = 0.5 * (Jc + Je)
    S = np.clip(S, 0.0, 1.0)
    np.fill_diagonal(S, 1.0)
    return S

# ---------- 메인 함수들 (Literal 없이 문자열 인자) ----------

def compute_S_matrix_soft_mcc(probs, labels, reduce="mean_over_clients", eps=1e-9):
    """
    Soft-MCC similarity.
    probs: (n_models, n_clients, n_items), labels: (n_clients, n_items)
    """
    n_models, n_clients, _ = probs.shape

    if reduce == "global":
        Sprob = labels[None, :, :] * probs + (1.0 - labels[None, :, :]) * (1.0 - probs)
        S_items_models = Sprob.reshape(n_models, -1).T
        S = _pairwise_soft_mcc_from_s(S_items_models, eps=eps)
        np.fill_diagonal(S, 1.0)
        return S

    S_acc = np.zeros((n_models, n_models), dtype=np.float64)
    for j in range(n_clients):
        s_j = labels[j][None, :] * probs[:, j, :] + (1.0 - labels[j][None, :]) * (1.0 - probs[:, j, :])
        S_acc += _pairwise_soft_mcc_from_s(s_j.T, eps=eps)
    S = S_acc / float(n_clients)
    np.fill_diagonal(S, 1.0)
    return S

def compute_S_matrix_kappa_or_mcc(probs, labels, method="kappa", threshold=0.5, reduce="mean_over_clients", eps=0):
    """
    Cohen's kappa or hard-MCC on binary correctness.
    """
    n_models, n_clients, _ = probs.shape
    y = (probs >= float(threshold)).astype(np.int8)
    correct = (y == labels[None, :, :]).astype(np.float64)

    if reduce == "global":
        C = correct.reshape(n_models, -1).T
        S = _pairwise_kappa_from_binary(C, eps=eps) if method == "kappa" else _pairwise_hard_mcc_from_binary(C, eps=eps)
        np.fill_diagonal(S, 1.0)
        return S

    S_acc = np.zeros((n_models, n_models), dtype=np.float64)
    for j in range(n_clients):
        Cj = correct[:, j, :].T
        S_acc += _pairwise_kappa_from_binary(Cj, eps=eps) if method == "kappa" else _pairwise_hard_mcc_from_binary(Cj, eps=eps)
    S = S_acc / float(n_clients)
    np.fill_diagonal(S, 1.0)
    return S

def compute_S_matrix_signed_margin_cosine(probs, labels, reduce="mean_over_clients"):
    """
    Signed-margin cosine similarity.
    """
    n_models, n_clients, _ = probs.shape
    twoq_minus1 = 2.0 * labels - 1.0
    twop_minus1 = 2.0 * probs - 1.0

    if reduce == "global":
        Z = (twoq_minus1[None, :, :] * twop_minus1).reshape(n_models, -1).T
        S = _pairwise_cosine_from_matrix(Z)
        np.fill_diagonal(S, 1.0)
        return S

    S_acc = np.zeros((n_models, n_models), dtype=np.float64)
    for j in range(n_clients):
        Zj = (twoq_minus1[j][None, :] * twop_minus1[:, j, :]).T
        S_acc += _pairwise_cosine_from_matrix(Zj)
    S = S_acc / float(n_clients)
    np.fill_diagonal(S, 1.0)
    return S

def compute_S_matrix_balanced_jaccard(probs, labels, threshold=0.5, reduce="mean_over_clients"):
    """
    Balanced Jaccard similarity over correctness patterns.
    """
    n_models, n_clients, _ = probs.shape
    y = (probs >= float(threshold)).astype(np.int8)
    correct = (y == labels[None, :, :]).astype(np.float64)

    if reduce == "global":
        C = correct.reshape(n_models, -1).T
        S = _pairwise_balanced_jaccard_from_binary(C)
        np.fill_diagonal(S, 1.0)
        return S

    S_acc = np.zeros((n_models, n_models), dtype=np.float64)
    for j in range(n_clients):
        Cj = correct[:, j, :].T
        S_acc += _pairwise_balanced_jaccard_from_binary(Cj)
    S = S_acc / float(n_clients)
    np.fill_diagonal(S, 1.0)
    return S

def compute_S_matrix(probs, labels, metric="soft_mcc", reduce="mean_over_clients", threshold=0.5, eps=1e-9):
    """
    Unified entry: metric in {"soft_mcc","kappa","hard_mcc","cosine","balanced_jaccard"}.
    """
    if metric == "soft_mcc":
        return compute_S_matrix_soft_mcc(probs, labels, reduce=reduce, eps=eps)
    if metric == "kappa":
        return compute_S_matrix_kappa_or_mcc(probs, labels, method="kappa", threshold=threshold, reduce=reduce, eps=eps)
    if metric == "hard_mcc":
        return compute_S_matrix_kappa_or_mcc(probs, labels, method="mcc", threshold=threshold, reduce=reduce, eps=eps)
    if metric == "cosine":
        return compute_S_matrix_signed_margin_cosine(probs, labels, reduce=reduce)
    if metric == "balanced_jaccard":
        return compute_S_matrix_balanced_jaccard(probs, labels, threshold=threshold, reduce=reduce)
    raise ValueError("Unknown metric: %s" % metric)


In [4]:
import numpy as np

def dict_to_arrays(probs_dict, labels_dict):
    """
    probs_dict: dict[int]->dict[int]->1D np.array (n_items,)
    labels_dict: dict[int]->1D np.array (n_items,)
    returns:
        P: (n_models, n_clients, n_items)
        L: (n_clients, n_items)
        model_keys, client_keys: 리스트(원복용)
    """
    model_keys  = sorted(probs_dict.keys())         # [1..M]
    client_keys = sorted(labels_dict.keys())        # [1..J]
    n_models, n_clients = len(model_keys), len(client_keys)

    # 기준 item 길이
    n_items = int(np.asarray(labels_dict[client_keys[0]]).shape[0])

    # L 쌓기
    L = np.stack([np.asarray(labels_dict[j], dtype=float) for j in client_keys], axis=0)
    assert L.shape == (n_clients, n_items), "labels shape mismatch"

    # P 쌓기
    P = np.zeros((n_models, n_clients, n_items), dtype=float)
    for mi, i in enumerate(model_keys):
        for cj, j in enumerate(client_keys):
            assert j in probs_dict[i], f"probs[{i}][{j}] missing"
            v = np.asarray(probs_dict[i][j], dtype=float)
            assert v.ndim == 1 and v.shape[0] == n_items, f"probs[{i}][{j}] length mismatch"
            P[mi, cj, :] = v

    return P, L, model_keys, client_keys



# 1) dict -> ndarray로 변환
P, L, model_keys, client_keys = dict_to_arrays(probs, labels)

# 2) 기존 함수 그대로 사용
S_matrix = compute_S_matrix(P, L, metric="hard_mcc", reduce="mean_over_clients")

print(S_matrix)

[[1.         0.47051336 0.62029763 ... 0.73102943 0.67831652 0.66667448]
 [0.47051336 1.         0.54799791 ... 0.58728572 0.59281471 0.60663756]
 [0.62029763 0.54799791 1.         ... 0.65524568 0.49302818 0.48322926]
 ...
 [0.73102943 0.58728572 0.65524568 ... 1.         0.64543384 0.67803143]
 [0.67831652 0.59281471 0.49302818 ... 0.64543384 1.         0.82786919]
 [0.66667448 0.60663756 0.48322926 ... 0.67803143 0.82786919 1.        ]]


In [5]:
import numpy as np
from typing import Optional, Tuple
from itertools import combinations

# ---------------------------
# k-medoids (PAM)
# ---------------------------
def _pam_k_medoids(
    D: np.ndarray,
    K: int,
    n_init: int = 16,
    max_iter: int = 200,
    rng: Optional[np.random.Generator] = None
) -> Tuple[np.ndarray, np.ndarray, float]:
    """
    Partitioning Around Medoids with multi-start.
    Returns (medoids_indices, cluster_labels, total_cost).
    """
    n = D.shape[0]
    rng = rng or np.random.default_rng(42)

    best_cost = np.inf
    best_meds = None
    best_labels = None

    for _ in range(n_init):
        meds = rng.choice(n, K, replace=False)
        labels_k = np.argmin(D[:, meds], axis=1)
        cur_cost = float(D[np.arange(n), meds[labels_k]].sum())

        for _ in range(max_iter):
            improved = False
            non_medoids = np.setdiff1d(np.arange(n), meds, assume_unique=True)

            for mi in range(K):
                for h in non_medoids:
                    new_meds = meds.copy()
                    new_meds[mi] = h
                    new_labels = np.argmin(D[:, new_meds], axis=1)
                    new_cost = float(D[np.arange(n), new_meds[new_labels]].sum())
                    if new_cost + 1e-12 < cur_cost:
                        meds = new_meds
                        labels_k = new_labels
                        cur_cost = new_cost
                        improved = True
                        break
                if improved:
                    break

            if not improved:
                break

        if cur_cost < best_cost:
            best_cost = cur_cost
            best_meds = meds
            best_labels = labels_k

    return best_meds, best_labels, best_cost


def reorder_medoids_and_labels(medoids: np.ndarray, labels: np.ndarray):
    """
    medoids를 오름차순으로 정렬하고, 그에 맞춰 labels도 재매핑.
    """
    order = np.argsort(medoids)      # old_idx -> 위치
    inv_perm = np.empty_like(order)  # inv_perm[old_idx] = new_idx
    inv_perm[order] = np.arange(len(order))
    new_medoids = medoids[order]
    new_labels  = inv_perm[labels]
    return new_medoids, new_labels


# ---------------------------
# Silhouette (all / macro / excl singletons)
# ---------------------------
def silhouette_macro_and_excl_from_D(D: np.ndarray, labels: np.ndarray):
    """
    D: (n,n) 거리행렬(대칭, diag=0)
    labels: 0..K-1
    반환: Silhouette 요약 + a(i)/b(i) 평균/벡터
    Keys:
      - silhouette_all:      (1/N) * sum_i s(i)
      - silhouette_macro:    (1/K) * sum_k \bar{s}_k,  \bar{s}_k = cluster k 평균 s(i)
      - silhouette_excl:     size>=2 포인트만 평균
      - sbar_by_cluster:     {k: \bar{s}_k}
      - singleton_ratio_clusters: 싱글톤 클러스터 비율
      - a_mean, b_mean, a_vec, b_vec, s_vec
    """
    n = D.shape[0]
    uniq = np.unique(labels)

    a = np.zeros(n, dtype=float)
    b = np.zeros(n, dtype=float)
    s = np.zeros(n, dtype=float)

    for k in uniq:
        idx_k = np.where(labels == k)[0]
        m = idx_k.size
        if m <= 1:
            if m == 1:
                # 싱글톤: s(i)=0으로 정의
                a[idx_k[0]] = 0.0
                b[idx_k[0]] = 0.0
                s[idx_k[0]] = 0.0
            continue

        # a(i): 같은 군집 평균거리
        a_i = D[np.ix_(idx_k, idx_k)].sum(axis=1) / (m - 1)

        # b(i): 가장 가까운 외부 군집 평균거리
        other_means = []
        for t in uniq:
            if t == k:
                continue
            idx_t = np.where(labels == t)[0]
            if idx_t.size == 0:
                continue
            other_means.append(D[np.ix_(idx_k, idx_t)].mean(axis=1))
        b_i = np.min(np.vstack(other_means), axis=0) if other_means else np.zeros_like(a_i)

        denom = np.maximum(a_i, b_i) + 1e-12
        s[idx_k] = (b_i - a_i) / denom
        a[idx_k] = a_i
        b[idx_k] = b_i

    # (1) pointwise 평균
    silhouette_all = float(np.mean(s)) if n > 0 else 0.0

    # (2) macro by cluster
    sbar_by_cluster = {}
    sbar_list = []
    singleton_clusters = 0
    for k in uniq:
        idx_k = np.where(labels == k)[0]
        if idx_k.size <= 1:
            sbar_by_cluster[int(k)] = 0.0
            sbar_list.append(0.0)
            if idx_k.size == 1:
                singleton_clusters += 1
        else:
            m = float(s[idx_k].mean())
            sbar_by_cluster[int(k)] = m
            sbar_list.append(m)
    silhouette_macro = float(np.mean(sbar_list)) if sbar_list else 0.0

    # (3) excl singletons
    mask_excl = np.zeros(n, dtype=bool)
    for k in uniq:
        idx_k = np.where(labels == k)[0]
        if idx_k.size >= 2:
            mask_excl[idx_k] = True
    silhouette_excl = float(np.mean(s[mask_excl])) if mask_excl.any() else 0.0

    return {
        "silhouette_all": silhouette_all,
        "silhouette_macro": silhouette_macro,
        "silhouette_excl": silhouette_excl,
        "sbar_by_cluster": sbar_by_cluster,
        "singleton_ratio_clusters": (singleton_clusters/len(uniq)) if len(uniq) else 0.0,
        "a_mean": float(np.mean(a)) if n > 0 else 0.0,
        "b_mean": float(np.mean(b)) if n > 0 else 0.0,
        "a_vec": a, "b_vec": b, "s_vec": s,
    }


# ---------------------------
# Intra/Inter (macro/micro), Nearest-Inter, Gini
# ---------------------------
def _gini_of_sizes(sizes):
    if len(sizes) == 0:
        return 0.0
    xs = np.sort(np.array(sizes, dtype=float))
    cum = np.cumsum(xs)
    K = len(xs)
    return float(1.0 - (2.0/(K*cum[-1]))*np.sum(cum) + 1.0/K)

def cluster_metrics_from_S_and_labels(S: np.ndarray, labels: np.ndarray):
    """
    S: (n,n) 유사도(대칭, diag≈1), labels: 0..K-1
    반환: intra_macro/micro, inter_macro/micro, nearest_inter(size-weighted), gini, etc.
    """
    n = S.shape[0]
    uniq = np.unique(labels)
    C = {int(k): np.where(labels == k)[0] for k in uniq}
    sizes = {k: C[k].size for k in C}

    # Intra_k
    intra_k = {}
    pair_sum, pair_cnt = 0.0, 0
    for k in uniq:
        idx = C[int(k)]
        if idx.size >= 2:
            vals = [S[i, v] for (i, v) in combinations(idx.tolist(), 2)]
            intra_k[int(k)] = float(np.mean(vals))
            pair_sum += float(np.sum(vals))
            pair_cnt += len(vals)
        else:
            intra_k[int(k)] = 0.0
    intra_macro = float(np.mean(list(intra_k.values()))) if intra_k else 0.0
    valid_intra = [intra_k[k] for k in intra_k if sizes[k] >= 2]
    intra_macro_excl = float(np.mean(valid_intra)) if valid_intra else 0.0
    intra_micro = (pair_sum / pair_cnt) if pair_cnt > 0 else 0.0

    # Inter_{k,u}
    inter_ku = {}
    cross_sum_total, cross_pairs_total = 0.0, 0
    for a in uniq:
        for b in uniq:
            if b <= a:
                continue
            ia, ib = C[int(a)], C[int(b)]
            if ia.size == 0 or ib.size == 0:
                inter_ku[(int(a), int(b))] = 0.0
                continue
            block = S[np.ix_(ia, ib)]
            inter_ku[(int(a), int(b))] = float(block.mean())
            cross_sum_total += float(block.sum())
            cross_pairs_total += block.size
    inter_macro = float(np.mean(list(inter_ku.values()))) if inter_ku else 0.0
    inter_micro = (cross_sum_total / cross_pairs_total) if cross_pairs_total > 0 else 0.0

    # Nearest-Inter (size-weighted)
    D = 1.0 - S
    nearest_terms, weights = [], []
    for k in uniq:
        k = int(k)
        nk = sizes[k]
        if nk == 0:
            continue
        best_u, best_d = None, np.inf
        for u in uniq:
            u = int(u)
            if u == k or sizes[u] == 0:
                continue
            d_mean = float(D[np.ix_(C[k], C[u])].mean())
            if d_mean < best_d:
                best_d, best_u = d_mean, u
        if best_u is None:
            continue
        s_mean = float(S[np.ix_(C[k], C[best_u])].mean())
        nearest_terms.append(nk * s_mean)
        weights.append(nk)
    nearest_inter = (sum(nearest_terms)/sum(weights)) if sum(weights) > 0 else 0.0

    gini = _gini_of_sizes(list(sizes.values()))
    effectiveK = sum(1 for k in sizes if sizes[k] >= 2)

    return {
        "intra_k": intra_k,
        "intra_macro": intra_macro,
        "intra_macro_excl": intra_macro_excl,
        "intra_micro": intra_micro,
        "inter_macro": inter_macro,
        "inter_micro": inter_micro,
        "nearest_inter": nearest_inter,     # ↑일수록 분리도 나쁨
        "cluster_sizes": sizes,
        "gini": gini,                       # ↑일수록 크기 불균형 심함
        "effectiveK_non_singleton": effectiveK,
        "inter_ku": inter_ku,
    }

# ====== (A) 라벨 기반 보조 함수들 ======
def r_med_from_labels(S: np.ndarray, D: np.ndarray, labels: np.ndarray) -> float:
    # 각 클러스터 medoid(라벨 기준) 구하고, 포인트별 '자기 medoid vs 타군집 medoid 최소거리' 비교
    uniq = np.unique(labels)
    C = {int(k): np.where(labels == k)[0] for k in uniq}
    meds = {}
    for k, idx in C.items():
        if idx.size == 0:
            meds[k] = None
            continue
        if idx.size == 1:
            meds[k] = idx[0]
        else:
            subD = D[np.ix_(idx, idx)]
            mlocal = idx[np.argmin(subD.sum(axis=1))]
            meds[k] = mlocal

    n = S.shape[0]
    violations = 0
    for i in range(n):
        ki = int(labels[i])
        mi = meds[ki]
        if mi is None:
            continue
        di_self = D[i, mi]
        best_other = np.inf
        for ku, mu in meds.items():
            if ku == ki or mu is None:
                continue
            if D[i, mu] < best_other:
                best_other = D[i, mu]
        if np.isfinite(best_other) and di_self > best_other + 1e-12:
            violations += 1
    return violations / float(n)


def M_D_from_labels(S: np.ndarray, D: np.ndarray, labels: np.ndarray) -> float:
    # Distance Margin = size-weighted NI_D - IntraDist_micro
    uniq = np.unique(labels)
    C = {int(k): np.where(labels == k)[0] for k in uniq}
    sizes = {k: C[k].size for k in C}

    # IntraDist_micro
    pair_cnt = sum((sizes[k]*(sizes[k]-1))//2 for k in sizes)
    if pair_cnt > 0:
        intra_sum = 0.0
        for k in sizes:
            idx = C[k]
            m = idx.size
            if m >= 2:
                triu = np.triu_indices(m, 1)
                intra_sum += float(S[np.ix_(idx, idx)][triu].sum())
        intra_micro = intra_sum / pair_cnt
    else:
        intra_micro = 0.0
    intraDist_micro = 1.0 - intra_micro

    # size-weighted NI_D
    Dmat = 1.0 - S
    num_sum = 0.0
    w_sum   = 0.0
    for k in sizes:
        idxk = C[k]
        if idxk.size == 0:
            continue
        best = np.inf
        for u in sizes:
            if u == k or sizes[u] == 0:
                continue
            d_mean = float(Dmat[np.ix_(idxk, C[u])].mean())
            if d_mean < best:
                best = d_mean
        if np.isfinite(best):
            num_sum += sizes[k] * best
            w_sum   += sizes[k]
    NI_D_size_weighted = (num_sum / w_sum) if w_sum > 0 else 0.0

    return NI_D_size_weighted - intraDist_micro


def MD_min_from_labels(S: np.ndarray, D: np.ndarray, labels: np.ndarray) -> float:
    # Worst-Cluster Margin = min_k [ NI_D(k) - Dbar_intra(Ck) ]
    uniq = np.unique(labels)
    C = {int(k): np.where(labels == k)[0] for k in uniq}
    sizes = {k: C[k].size for k in C}
    Dmat = 1.0 - S

    worst = np.inf
    for k in sizes:
        idxk = C[k]
        m = idxk.size
        if m <= 1:
            continue
        # Dbar_intra(Ck)
        triu = np.triu_indices(m, 1)
        Dintra = float(Dmat[np.ix_(idxk, idxk)][triu].mean())
        # NI_D(k)
        best = np.inf
        for u in sizes:
            if u == k or sizes[u] == 0:
                continue
            d_mean = float(Dmat[np.ix_(idxk, C[u])].mean())
            if d_mean < best:
                best = d_mean
        if np.isfinite(best):
            margin_k = best - Dintra
            if margin_k < worst:
                worst = margin_k
    return 0.0 if not np.isfinite(worst) else float(worst)


# ====== (B) 널-보정(Z-score) 유틸 ======
def null_standardize_metric_from_labels(S: np.ndarray,
                                        D: np.ndarray,
                                        labels: np.ndarray,
                                        metric_fn,
                                        R: int = 500,
                                        rng: Optional[np.random.Generator] = None):
    """
    labels의 클러스터 크기 분포(n1..nK)는 유지하고, 라벨만 R회 셔플 → metric의 null 분포를 만들고
    Z = (obs - mean) / std 반환
    """
    rng = rng or np.random.default_rng(123)
    n = labels.shape[0]
    uniq, counts = np.unique(labels, return_counts=True)
    obs = float(metric_fn(S, D, labels))

    null_vals = []
    base = np.arange(n)
    for _ in range(R):
        rng.shuffle(base)
        # 사이즈 분포 유지된 새 라벨 만들기
        new_labels = np.empty(n, dtype=int)
        offset = 0
        for k, c in zip(uniq, counts):
            new_labels[base[offset:offset+c]] = k
            offset += c
        null_vals.append(float(metric_fn(S, D, new_labels)))

    null_vals = np.array(null_vals, dtype=float)
    mu = null_vals.mean()
    sd = null_vals.std(ddof=1) + 1e-12
    z  = (obs - mu) / sd
    return {"obs": obs, "null_mean": mu, "null_std": sd, "z": z}


# ====== (C) PAM 라벨 결과를 한 번에 요약 ======
def summarize_pam_result(name, S, D, medoids, labels, cost, R=500):
    meds_sorted, lbls_sorted = reorder_medoids_and_labels(medoids, labels)

    print(f"\n### [PAM] {name}")
    print("Best medoids (0-based, sorted):", (np.asarray(meds_sorted) + 1).tolist())

    sil = silhouette_macro_and_excl_from_D(D, lbls_sorted)
    print(f"Silhouette (all)   : {sil['silhouette_all']:.6f}")
    print(f"Silhouette (macro) : {sil['silhouette_macro']:.6f}")
    print(f"Silhouette (excl1) : {sil['silhouette_excl']:.6f}")
    print(f"  singleton ratio  : {sil['singleton_ratio_clusters']:.3f}")

    var = cluster_metrics_from_S_and_labels(S, lbls_sorted)
    print(f"Intra_micro        : {var['intra_micro']:.6f}")
    print(f"Inter_micro        : {var['inter_micro']:.6f}")
    print(f"Nearest-Inter (S)  : {var['nearest_inter']:.6f}   # ↑일수록 분리도 나쁨")
    print(f"Sizes              : {var['cluster_sizes']}")
    print(f"Gini(size)         : {var['gini']:.4f}")
    print(f"PAM Cost           : {float(cost):.6f}")

    extra = extra_metrics_from_S_and_labels(S, D, lbls_sorted)
    print("[Proposed metrics - raw]")
    print(f"  r_neg                    : {extra['r_neg']:.3f}   (↑ 나쁨)")
    print(f"  Burden                   : {extra['burden']:.4f} (↑ 나쁨)")
    print(f"  M_D                      : {extra['M_D']:.6f}    (↑ 좋음)")
    print(f"  NI_D (size-weighted)     : {extra['NI_D_size_weighted']:.6f} (↑ 좋음)")
    print(f"  NI_D_min (worst cluster) : {extra['NI_D_min']:.6f} (↑ 좋음)")
    print(f"  IntraDist_micro          : {extra['IntraDist_micro']:.6f}")

    rmed_raw = r_med_from_labels(S, D, lbls_sorted)
    rmed_ns  = null_standardize_metric_from_labels(S, D, lbls_sorted, r_med_from_labels, R=R)
    print("[Medoid Violation]")
    print(f"  r_med (raw)              : {rmed_raw:.4f}  (↓ 좋음)")
    print(f"  r_med Z (null-std, +good): {-rmed_ns['z']:.3f}  # Z는 ↑가 좋음이라 부호 반전")

    md_raw = M_D_from_labels(S, D, lbls_sorted)
    md_ns  = null_standardize_metric_from_labels(S, D, lbls_sorted, M_D_from_labels, R=R)
    print("[Distance Margin]")
    print(f"  M_D (raw)                : {md_raw:.6f}  (↑ 좋음)")
    print(f"  M_D Z (null-std)         : {md_ns['z']:.3f}  (↑ 좋음)")

    mdmin_raw = MD_min_from_labels(S, D, lbls_sorted)
    mdmin_ns  = null_standardize_metric_from_labels(S, D, lbls_sorted, MD_min_from_labels, R=R)
    print("[Worst-Cluster Margin]")
    print(f"  M_Dmin (raw)             : {mdmin_raw:.6f}  (↑ 좋음)")
    print(f"  M_Dmin Z (null-std)      : {mdmin_ns['z']:.3f}  (↑ 좋음)")




def _nearest_inter_dist_terms_from_labels(S: np.ndarray, labels: np.ndarray):
    """
    각 클러스터 k에 대해 '가장 가까운 외부 군집'까지의 평균거리( Dbar(Ck,Cu) )를 반환.
    - 반환:
        per_cluster_minD: dict{k -> min_u!=k Dbar(Ck,Cu)}
        size_weighted_NI_D:   sum_k n_k * minD_k / sum_k n_k     (크기 가중 평균)
        min_over_clusters:    min_k minD_k                       (가장 위험한 군집의 최근접거리)
    """
    D = 1.0 - S
    uniq = np.unique(labels)
    C = {int(k): np.where(labels == k)[0] for k in uniq}
    sizes = {k: C[k].size for k in C}

    per_cluster_minD = {}
    num_sum = 0.0
    w_sum   = 0.0
    min_over_clusters = np.inf

    for k in uniq:
        k = int(k)
        idxk = C[k]
        if idxk.size == 0:
            per_cluster_minD[k] = np.nan
            continue
        best = np.inf
        for u in uniq:
            u = int(u)
            if u == k or sizes[u] == 0:
                continue
            d_mean = float(D[np.ix_(idxk, C[u])].mean())
            if d_mean < best:
                best = d_mean
        if np.isfinite(best):
            per_cluster_minD[k] = best
            # 크기 가중 최근접-Inter(거리)
            num_sum += sizes[k] * best
            w_sum   += sizes[k]
            if best < min_over_clusters:
                min_over_clusters = best
        else:
            per_cluster_minD[k] = np.nan

    size_weighted_NI_D = (num_sum / w_sum) if w_sum > 0 else 0.0
    if not np.isfinite(min_over_clusters):
        min_over_clusters = 0.0
    return per_cluster_minD, size_weighted_NI_D, float(min_over_clusters)


def extra_metrics_from_S_and_labels(S: np.ndarray, D: np.ndarray, labels: np.ndarray):
    """
    위 4개 지표를 한 번에 계산.
    필요한 중간값은 기존 함수들 출력과 일관되게 재사용/재계산.
    반환 dict:
      r_neg, burden, M_D, NI_D_size_weighted, NI_D_min, IntraDist_micro
    """
    # 1) 실루엣 s(i) : 이미 있는 silhouette 함수에서 s_vec을 얻는게 가장 좋음.
    # 만약 silhouette_macro_and_excl_from_D 가 s_vec을 반환하지 않으면 여기서 직접 계산:
    n = D.shape[0]
    uniq = np.unique(labels)
    s = np.zeros(n, dtype=float)

    for k in uniq:
        idxk = np.where(labels == k)[0]
        if idxk.size <= 1:
            if idxk.size == 1:
                s[idxk[0]] = 0.0
            continue
        # a(i): 같은 군집 평균거리
        a_i = D[np.ix_(idxk, idxk)].sum(axis=1) / (idxk.size - 1)
        # b(i): 가장 가까운 외부군집 평균거리
        other_means = []
        for u in uniq:
            if u == k: 
                continue
            idxu = np.where(labels == u)[0]
            if idxu.size == 0:
                continue
            other_means.append(D[np.ix_(idxk, idxu)].mean(axis=1))
        b_i = np.min(np.vstack(other_means), axis=0) if other_means else np.zeros_like(a_i)
        denom = np.maximum(a_i, b_i) + 1e-12
        s[idxk] = (b_i - a_i) / denom

    # 2) r_neg, burden
    r_neg  = float(np.mean(s < 0))
    burden = float(np.maximum(0.0, -s).mean())

    # 3) Intra micro (유사도) -> 거리 스케일로 변환
    #    labels 버전 intra_micro는 기존 cluster_metrics_from_S_and_labels에서 이미 계산함.
    #    여기서는 독립적으로 계산:
    C = {int(k): np.where(labels == k)[0] for k in uniq}
    sizes = {k: C[k].size for k in C}
    pair_cnt = sum((sizes[k]*(sizes[k]-1))//2 for k in sizes)
    if pair_cnt > 0:
        intra_sum = 0.0
        for k in sizes:
            idx = C[k]
            m = idx.size
            if m >= 2:
                triu = np.triu_indices(m, 1)
                intra_sum += float(S[np.ix_(idx, idx)][triu].sum())
        intra_micro = intra_sum / pair_cnt
    else:
        intra_micro = 0.0
    intraDist_micro = 1.0 - intra_micro

    # 4) 최근접-Inter 거리 (크기 가중 평균) & 최솟값(최악 군집)
    _, NI_D_size_weighted, NI_D_min = _nearest_inter_dist_terms_from_labels(S, labels)

    # 5) Distance Margin
    M_D = NI_D_size_weighted - intraDist_micro

    return {
        "r_neg": r_neg,
        "burden": burden,
        "M_D": M_D,
        "NI_D_size_weighted": NI_D_size_weighted,
        "NI_D_min": NI_D_min,
        "IntraDist_micro": intraDist_micro,
    }



def clusters_by_medoid(medoids, labels, index_base=0):
    """
    medoids: (K,) 0-based 데이터 인덱스 (ex. [15,16,20])
    labels : (N,) 각 포인트의 클러스터 라벨(0..K-1)
    index_base: 0이면 0-based로, 1이면 1-based로 반환

    return: dict[int, list[int]]
            { <medoid_index>: [<멤버 인덱스들(오름차순)>] }
    """
    medoids = np.asarray(medoids, dtype=int)
    labels  = np.asarray(labels, dtype=int)

    # 각 포인트 i가 실제 어느 medoid(데이터 인덱스)에 배정되었는지
    assigned_medoid_idx = medoids[labels]   # (N,), 값은 0-based medoid 인덱스

    clusters = {}
    for m in np.sort(medoids):              # 키는 medoid를 오름차순으로
        members = np.where(assigned_medoid_idx == m)[0]  # 0-based 멤버 인덱스
        if index_base == 1:
            key = int(m) + 1
            val = (members + 1).tolist()
        else:
            key = int(m)
            val = members.tolist()
        clusters[key] = sorted(val)
    return clusters



In [6]:
S = S_matrix
D = 1.0 - S
np.fill_diagonal(D, 0.0)
for K in [3]:
    meds, lbls, pam_cost = _pam_k_medoids(D, K=K, n_init=24, max_iter=1000)
    summarize_pam_result(f"K={K}", S, D, meds, lbls, pam_cost, R=500)

    # 1-based로 사전 만들기
    clusters = clusters_by_medoid(meds, lbls, index_base=1)

    # 보기 좋은 출력
    print("[Clusters by medoid] (1-based indices)")
    print(f"  Best medoids (1-based, sorted): {sorted((np.array(meds)+1).tolist())}")
    for m in sorted(clusters.keys()):
        print(f"  medoid {m:>2d}: {clusters[m]}")

    # (선택) 검증: 모든 멤버 수 합이 53인지
    assert sum(len(v) for v in clusters.values()) == 53



### [PAM] K=3
Best medoids (0-based, sorted): [16, 17, 21]
Silhouette (all)   : 0.206605
Silhouette (macro) : 0.221856
Silhouette (excl1) : 0.206605
  singleton ratio  : 0.000
Intra_micro        : 0.705440
Inter_micro        : 0.567429
Nearest-Inter (S)  : 0.615644   # ↑일수록 분리도 나쁨
Sizes              : {0: 21, 1: 23, 2: 9}
Gini(size)         : 0.1761
PAM Cost           : 11.659054
[Proposed metrics - raw]
  r_neg                    : 0.094   (↑ 나쁨)
  Burden                   : 0.0058 (↑ 나쁨)
  M_D                      : 0.089796    (↑ 좋음)
  NI_D (size-weighted)     : 0.384356 (↑ 좋음)
  NI_D_min (worst cluster) : 0.366676 (↑ 좋음)
  IntraDist_micro          : 0.294560
[Medoid Violation]
  r_med (raw)              : 0.0000  (↓ 좋음)
  r_med Z (null-std, +good): 9.209  # Z는 ↑가 좋음이라 부호 반전
[Distance Margin]
  M_D (raw)                : 0.089796  (↑ 좋음)
  M_D Z (null-std)         : 10.858  (↑ 좋음)
[Worst-Cluster Margin]
  M_Dmin (raw)             : 0.066264  (↑ 좋음)
  M_Dmin Z (null-std)      : 8.93

In [7]:
import numpy as np
from itertools import combinations

# --- 기존: dict -> assignment (그대로 사용) ---
def _dict_clusters_to_assignment(clusters_dict, n=53, one_based=True):
    A = -np.ones(n, dtype=int)
    for k, members in clusters_dict.items():
        for idx in members:
            i = idx - 1 if one_based else idx
            A[i] = k
    if np.any(A < 0):
        missing = np.where(A < 0)[0]
        raise ValueError(f"할당 누락 인덱스(0-based): {missing.tolist()}")
    return A

# --- 기존: 주어진 클러스터에 대해 Medoid & Cost (그대로 사용) ---
def compute_medoids_and_cost(S_star: np.ndarray, clusters_dict: dict, one_based=True, fixed_medoids=None):
    S = np.asarray(S_star, dtype=float)
    n = S.shape[0]
    assert S.shape[0] == S.shape[1], "S* must be square"
    np.fill_diagonal(S, 1.0)
    D = 1.0 - S
    np.fill_diagonal(D, 0.0)

    C = {k: np.array([m-1 if one_based else m for m in members], dtype=int)
         for k, members in clusters_dict.items()}
    ks = sorted(C.keys())

    medoids_0 = {}
    for k in ks:
        members = C[k]
        if members.size == 0:
            medoids_0[k] = None
            continue
        if fixed_medoids is not None and k in fixed_medoids:
            m_idx = fixed_medoids[k]
            m0 = (m_idx - 1) if one_based else m_idx
            medoids_0[k] = m0
        else:
            if members.size == 1:
                medoids_0[k] = members[0]
            else:
                intra_dists = D[np.ix_(members, members)]
                totals = intra_dists.sum(axis=1)
                medoids_0[k] = members[np.argmin(totals)]

    cost_per_cluster = {}
    total_cost = 0.0
    for k in ks:
        members = C[k]
        if members.size == 0:
            cost_per_cluster[k] = 0.0
            continue
        m0 = medoids_0[k]
        dists = D[members, m0]
        c = float(dists.sum())
        cost_per_cluster[k] = c
        total_cost += c

    medoids_1 = {k: (None if medoids_0[k] is None else medoids_0[k] + 1) for k in ks}
    return medoids_1, float(total_cost), cost_per_cluster

# ---------------------------
# Silhouette (all / macro / excl) — dict 클러스터 입력용
# ---------------------------
def silhouette_from_dict_D(D: np.ndarray, clusters_dict: dict, one_based=True):
    """
    반환 키를 PAM 버전과 동일하게:
      silhouette_all, silhouette_macro, silhouette_excl,
      sbar_by_cluster, singleton_ratio_clusters,
      a_mean, b_mean, a_vec, b_vec, s_vec
    """
    n = D.shape[0]
    ks = sorted(clusters_dict.keys())
    # 0/1-based 멤버 인덱스
    C = {k: np.array([m-1 if one_based else m for m in clusters_dict[k]], dtype=int) for k in ks}

    a = np.zeros(n, dtype=float)
    b = np.zeros(n, dtype=float)
    s = np.zeros(n, dtype=float)

    for k in ks:
        idx_k = C[k]
        m = idx_k.size
        if m <= 1:
            if m == 1:
                a[idx_k[0]] = 0.0
                b[idx_k[0]] = 0.0
                s[idx_k[0]] = 0.0
            continue
        # a(i)
        a_i = D[np.ix_(idx_k, idx_k)].sum(axis=1) / (m - 1)
        # b(i)
        other_means = []
        for u in ks:
            if u == k or C[u].size == 0:
                continue
            other_means.append(D[np.ix_(idx_k, C[u])].mean(axis=1))
        b_i = np.min(np.vstack(other_means), axis=0) if other_means else np.zeros_like(a_i)
        denom = np.maximum(a_i, b_i) + 1e-12
        s[idx_k] = (b_i - a_i) / denom
        a[idx_k] = a_i
        b[idx_k] = b_i

    # (1) 포인트 평균
    silhouette_all = float(np.mean(s)) if n > 0 else 0.0

    # (2) 매크로(클러스터 평균의 평균)
    sbar_by_cluster, sbar_list, singleton_clusters = {}, [], 0
    for k in ks:
        idx_k = C[k]
        if idx_k.size <= 1:
            sbar_by_cluster[k] = 0.0
            sbar_list.append(0.0)
            if idx_k.size == 1:
                singleton_clusters += 1
        else:
            val = float(s[idx_k].mean())
            sbar_by_cluster[k] = val
            sbar_list.append(val)
    silhouette_macro = float(np.mean(sbar_list)) if sbar_list else 0.0

    # (3) 싱글톤 제외(포인트 단위)
    mask_excl = np.zeros(n, dtype=bool)
    for k in ks:
        if C[k].size >= 2:
            mask_excl[C[k]] = True
    silhouette_excl = float(np.mean(s[mask_excl])) if mask_excl.any() else 0.0

    return {
        "silhouette_all": silhouette_all,
        "silhouette_macro": silhouette_macro,
        "silhouette_excl": silhouette_excl,
        "sbar_by_cluster": sbar_by_cluster,
        "singleton_ratio_clusters": (singleton_clusters/len(ks)) if len(ks) else 0.0,
        "a_mean": float(np.mean(a)) if n > 0 else 0.0,
        "b_mean": float(np.mean(b)) if n > 0 else 0.0,
        "a_vec": a, "b_vec": b, "s_vec": s,
    }

# ---------------------------
# Intra/Inter (macro/micro), Nearest-Inter, Gini — dict 입력용
# ---------------------------
def intra_inter_from_dict_S(S_star: np.ndarray, clusters_dict: dict, one_based=True):
    """
    반환 키를 PAM 버전과 맞춤:
      intra_macro, intra_macro_excl, intra_micro,
      inter_macro, inter_micro, nearest_inter,
      cluster_sizes, gini, effectiveK_non_singleton, inter_ku, intra_k
    """
    S = np.asarray(S_star, dtype=float)
    n = S.shape[0]
    np.fill_diagonal(S, 1.0)
    D = 1.0 - S
    np.fill_diagonal(D, 0.0)

    ks = sorted(clusters_dict.keys())
    C = {k: np.array([m-1 if one_based else m for m in clusters_dict[k]], dtype=int) for k in ks}
    sizes = {k: C[k].size for k in ks}

    # Intra_k
    intra_k = {}
    sum_pairs, cnt_pairs = 0.0, 0
    for k in ks:
        idx = C[k]
        m = idx.size
        if m >= 2:
            block = S[np.ix_(idx, idx)]
            triu = block[np.triu_indices(m, 1)]
            val = float(triu.mean()) if triu.size > 0 else 0.0
            intra_k[k] = val
            sum_pairs += float(triu.sum())
            cnt_pairs += triu.size
        else:
            intra_k[k] = 0.0
    intra_macro = float(np.mean(list(intra_k.values()))) if intra_k else 0.0
    valid_intra = [intra_k[k] for k in ks if sizes[k] >= 2]
    intra_macro_excl = float(np.mean(valid_intra)) if valid_intra else 0.0
    intra_micro = (sum_pairs / cnt_pairs) if cnt_pairs > 0 else 0.0

    # Inter
    inter_ku = {}
    inter_list, cross_sum, cross_cnt = [], 0.0, 0
    for i, k in enumerate(ks):
        for u in ks[i+1:]:
            ck, cu = C[k], C[u]
            if ck.size == 0 or cu.size == 0:
                inter_ku[(k, u)] = 0.0
                inter_list.append(0.0)
                continue
            block = S[np.ix_(ck, cu)]
            val = float(block.mean())
            inter_ku[(k, u)] = val
            inter_list.append(val)
            cross_sum += float(block.sum())
            cross_cnt += block.size
    inter_macro = float(np.mean(inter_list)) if inter_list else 0.0
    inter_micro = (cross_sum / cross_cnt) if cross_cnt > 0 else 0.0

    # Nearest-Inter (size-weighted)
    nearest_terms, weights = [], []
    for k in ks:
        ck = C[k]
        nk = ck.size
        if nk == 0:
            continue
        best_d, best_u = np.inf, None
        for u in ks:
            if u == k or C[u].size == 0:
                continue
            d_mean = float(D[np.ix_(ck, C[u])].mean())
            if d_mean < best_d:
                best_d, best_u = d_mean, u
        if best_u is None:
            continue
        s_mean = float(S[np.ix_(ck, C[best_u])].mean())
        nearest_terms.append(nk * s_mean)
        weights.append(nk)
    nearest_inter = (sum(nearest_terms) / sum(weights)) if sum(weights) > 0 else 0.0

    # Gini (표준 이산형 근사식)
    sizes_arr = np.array(list(sizes.values()), dtype=float)
    if sizes_arr.sum() > 0:
        xs = np.sort(sizes_arr)
        cum = np.cumsum(xs)
        K = len(xs)
        gini = float(1.0 - (2.0/(K*cum[-1]))*np.sum(cum) + 1.0/K)
    else:
        gini = 0.0
    effectiveK = int(np.sum(sizes_arr >= 2))

    return {
        "intra_k": intra_k,
        "intra_macro": intra_macro,
        "intra_macro_excl": intra_macro_excl,
        "intra_micro": intra_micro,
        "inter_macro": inter_macro,
        "inter_micro": inter_micro,
        "nearest_inter": nearest_inter,
        "cluster_sizes": sizes,
        "gini": gini,
        "effectiveK_non_singleton": effectiveK,
        "inter_ku": inter_ku,
    }

# ---------------------------
# 통일 출력 포맷 (PAM 버전과 동일 순서/용어)
# ---------------------------
def evaluate_and_print_given_clusters(S_star, D, clusters_dict, name="clusters", one_based=True, floatfmt=".6f"):
    sil = silhouette_from_dict_D(D, clusters_dict, one_based=one_based)
    var = intra_inter_from_dict_S(S_star, clusters_dict, one_based=one_based)
    medoids, cost, _ = compute_medoids_and_cost(S_star, clusters_dict, one_based=one_based)

    K = len(clusters_dict)
    print(f"\n=== {name} (K={K}) ===")
    # Silhouette
    print(f"Silhouette (all points avg):       {sil['silhouette_all']:{floatfmt}}")
    print(f"Silhouette (macro by cluster):     {sil['silhouette_macro']:{floatfmt}}")
    print(f"  mean a(i): {sil['a_mean']:{floatfmt}}, mean b(i): {sil['b_mean']:{floatfmt}}")
    print(f"  s̄_k by cluster: {sil['sbar_by_cluster']}")
    print(f"  singleton ratio (clusters): {sil['singleton_ratio_clusters']:.3f}")
    print(f"Silhouette (excl singleton clusters):      {sil['silhouette_excl']:{floatfmt}}")

    # Intra / Inter / Nearest-Inter / Gini
    print(f"Intra macro (incl singleton clusters):   {var['intra_macro']:{floatfmt}}")
    print(f"Intra macro excl singleton clusters:       {var['intra_macro_excl']:{floatfmt}}")
    print(f"Intra micro (pair-weighted):       {var['intra_micro']:{floatfmt}}")
    print(f"Inter macro:                        {var['inter_macro']:{floatfmt}}")
    print(f"Inter micro (pair-weighted):        {var['inter_micro']:{floatfmt}}")
    print(f"Nearest-Inter (size-weighted):      {var['nearest_inter']:{floatfmt}}  <-- 값↑ ⇒ 분리도 나쁨/경계 혼선")
    print(f"Cluster sizes: {var['cluster_sizes']}")
    print(f"Gini(size): {var['gini']:.4f} | effective K(>=2): {var['effectiveK_non_singleton']}")

    # Medoid & Cost (주어진 클러스터 기준)
    print(f"Medoids (1-based): {medoids}")
    print(f"Cost: {cost:{floatfmt}}")


    extra = extra_metrics_from_S_and_clusters(S_star, clusters_dict, one_based=one_based)
    print(f"[Proposed metrics]")
    print(f"  Negative-Silhouette Rate r_neg:  {extra['r_neg']:.3f}   (↑ 나쁨)")
    print(f"  Neg-Silhouette Burden B:         {extra['burden']:.4f} (↑ 나쁨)")
    print(f"  Distance Margin M_D:             {extra['M_D']:.6f}     (↓ 나쁨)")
    print(f"  NI_D (size-weighted):            {extra['NI_D_size_weighted']:.6f} (↑ 좋음)")
    print(f"  NI_D_min (worst cluster):        {extra['NI_D_min']:.6f} (↑ 좋음)")


def compute_silhouette_intra_inter(S_star: np.ndarray, clusters_dict: dict, one_based: bool=True):
    S = np.asarray(S_star, dtype=float)
    np.fill_diagonal(S, 1.0)
    D = 1.0 - S
    np.fill_diagonal(D, 0.0)

    sil = silhouette_from_dict_D(D, clusters_dict, one_based=one_based)
    # 최소한 extra_metrics가 쓰는 키만 반환
    return {
        "silhouette_mean": sil["silhouette_all"],
        "a_vec": sil["a_vec"],
        "b_vec": sil["b_vec"],
        "s_vec": sil["s_vec"],
    }



def extra_metrics_from_S_and_clusters(S: np.ndarray, clusters_dict: dict, one_based=True):
    """
    dict형 클러스터가 주어졌을 때 4개 지표 계산.
    (compute_silhouette_intra_inter를 써서 s_vec/assignment를 재활용)
    """
    S = np.asarray(S, dtype=float)
    n = S.shape[0]
    D = 1.0 - S
    np.fill_diagonal(D, 0.0)

    # s_vec 얻기
    res_sil = compute_silhouette_intra_inter(S, clusters_dict, one_based=one_based)
    s = res_sil["s_vec"]  # 이미 0/싱글톤 처리된 벡터

    r_neg  = float(np.mean(s < 0))
    burden = float(np.maximum(0.0, -s).mean())

    # 사이즈, 인덱스
    ks = sorted(clusters_dict.keys())
    C = {k: np.array([m-1 if one_based else m for m in clusters_dict[k]], dtype=int) for k in ks}
    sizes = {k: C[k].size for k in ks}

    # Intra micro (유사도) -> 거리 스케일
    pair_cnt = sum((sizes[k]*(sizes[k]-1))//2 for k in ks)
    if pair_cnt > 0:
        intra_sum = 0.0
        for k in ks:
            idx = C[k]
            m = idx.size
            if m >= 2:
                triu = np.triu_indices(m, 1)
                intra_sum += float(S[np.ix_(idx, idx)][triu].sum())
        intra_micro = intra_sum / pair_cnt
    else:
        intra_micro = 0.0
    intraDist_micro = 1.0 - intra_micro

    # 최근접-Inter 거리 (크기 가중 평균) & 최솟값
    num_sum = 0.0
    w_sum   = 0.0
    worst_min = np.inf
    for k in ks:
        idxk = C[k]
        if idxk.size == 0:
            continue
        best = np.inf
        for u in ks:
            if u == k or C[u].size == 0:
                continue
            d_mean = float(D[np.ix_(idxk, C[u])].mean())
            if d_mean < best:
                best = d_mean
        if np.isfinite(best):
            num_sum += sizes[k]*best
            w_sum   += sizes[k]
            if best < worst_min:
                worst_min = best
    NI_D_size_weighted = (num_sum / w_sum) if w_sum > 0 else 0.0
    NI_D_min = worst_min if np.isfinite(worst_min) else 0.0

    M_D = NI_D_size_weighted - intraDist_micro

    return {
        "r_neg": r_neg,
        "burden": burden,
        "M_D": M_D,
        "NI_D_size_weighted": NI_D_size_weighted,
        "NI_D_min": NI_D_min,
        "IntraDist_micro": intraDist_micro,
    }



def add_null_block_for_dict(name, S, D, clusters_dict, one_based=True, R=500):
    """dict 클러스터에 대해 raw와 Z(null-standardized) 둘 다 출력"""
    labels = _dict_clusters_to_assignment(clusters_dict, n=S.shape[0], one_based=one_based)

    # raw 값 직접 계산
    rmed_raw  = r_med_from_labels(S, D, labels)
    md_raw    = M_D_from_labels(S, D, labels)
    mdmin_raw = MD_min_from_labels(S, D, labels)

    # null 표준화 (같은 (n1..nK) 분포 유지)
    rmed_ns  = null_standardize_metric_from_labels(S, D, labels, r_med_from_labels,  R=R)
    md_ns    = null_standardize_metric_from_labels(S, D, labels, M_D_from_labels,    R=R)
    mdmin_ns = null_standardize_metric_from_labels(S, D, labels, MD_min_from_labels, R=R)

    # 방향 통일: r_med는 ↓가 좋음 → Z에 부호반전해서 ↑가 좋음으로 보고
    rmed_z_good = -rmed_ns['z']

    print(f"[Null-standardized add-on] {name}")
    print(f"[Medoid Violation]")
    print(f"  r_med (raw)              : {rmed_raw:.4f}  (↓ 좋음)")
    print(f"  r_med Z (null-std, +good): {rmed_z_good:.3f}  (↑ 좋음)")
    print(f"[Distance Margin]")
    print(f"  M_D (raw)                : {md_raw:.6f}  (↑ 좋음)")
    print(f"  M_D Z (null-std)         : {md_ns['z']:.3f}  (↑ 좋음)")
    print(f"[Worst-Cluster Margin]")
    print(f"  M_Dmin (raw)             : {mdmin_raw:.6f}  (↑ 좋음)")
    print(f"  M_Dmin Z (null-std)      : {mdmin_ns['z']:.3f}  (↑ 좋음)")



In [8]:
c1_opt = {
    0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53],
}


c3_opt = {
    0: [1, 6, 10, 13, 15, 16, 19, 20, 22, 26, 28, 29, 34, 39, 40, 43, 45, 46, 47, 48, 51],
    1: [2, 4, 7, 8, 9, 11, 12, 14, 17, 18, 23, 24, 25, 27, 30, 33, 35, 38, 42, 44, 49, 52, 53],
    2: [3, 5, 21, 31, 32, 36, 37, 41, 50],
}



evaluate_and_print_given_clusters(S_matrix, D, c1_opt,  name="c1_opt",  one_based=True)
add_null_block_for_dict("c1_opt", S_matrix, D, c1_opt, one_based=True, R=500)

evaluate_and_print_given_clusters(S_matrix, D, c3_opt,  name="c3_opt",  one_based=True)
add_null_block_for_dict("c3_opt", S_matrix, D, c3_opt, one_based=True, R=500)







=== c1_opt (K=1) ===
Silhouette (all points avg):       -1.000000
Silhouette (macro by cluster):     -1.000000
  mean a(i): 0.382595, mean b(i): 0.000000
  s̄_k by cluster: {0: -0.999999999997357}
  singleton ratio (clusters): 0.000
Silhouette (excl singleton clusters):      -1.000000
Intra macro (incl singleton clusters):   0.617405
Intra macro excl singleton clusters:       0.617405
Intra micro (pair-weighted):       0.617405
Inter macro:                        0.000000
Inter micro (pair-weighted):        0.000000
Nearest-Inter (size-weighted):      0.000000  <-- 값↑ ⇒ 분리도 나쁨/경계 혼선
Cluster sizes: {0: 53}
Gini(size): 0.0000 | effective K(>=2): 1
Medoids (1-based): {0: 20}
Cost: 16.412988
[Proposed metrics]
  Negative-Silhouette Rate r_neg:  1.000   (↑ 나쁨)
  Neg-Silhouette Burden B:         1.0000 (↑ 나쁨)
  Distance Margin M_D:             -0.382595     (↓ 나쁨)
  NI_D (size-weighted):            0.000000 (↑ 좋음)
  NI_D_min (worst cluster):        0.000000 (↑ 좋음)
[Null-standardized add-on]

In [9]:
def labels_from_clusters_dict(clusters: dict, n: int, *, one_based: bool=True) -> np.ndarray:
    """
    clusters: {cluster_id(0..K-1): [members]}
    members가 1-based면 one_based=True로 주면 됩니다.
    """
    lbls = np.empty(n, dtype=int)
    for k, members in clusters.items():
        members = np.asarray(members, dtype=int)
        if one_based:
            members = members - 1
        lbls[members] = int(k)
    return lbls

def per_client_ab_df(D: np.ndarray, labels: np.ndarray, *, index_base: int = 1) -> pd.DataFrame:
    """
    각 클라이언트 i에 대한 a(i), b(i), s(i)와 cluster 라벨을 DataFrame으로 반환.
    """
    sil = silhouette_macro_and_excl_from_D(D, labels)  # 이미 a_vec, b_vec, s_vec 제공
    a, b, s = sil["a_vec"], sil["b_vec"], sil["s_vec"]
    idx = np.arange(labels.shape[0]) + index_base
    df = pd.DataFrame({
        "client": idx,
        "cluster": labels,   # 0..K-1
        "a(i)": a,
        "b(i)": b,
        "s(i)": s,
    }).set_index("client").sort_index()
    return df

def medoids_from_labels(D: np.ndarray, labels: np.ndarray) -> np.ndarray:
    """
    labels(0..K-1) 기준 각 클러스터의 메도이드를 0-based 데이터 인덱스로 반환.
    """
    uniq = np.unique(labels)
    meds = np.empty(len(uniq), dtype=int)
    for k in uniq:
        idx = np.where(labels == k)[0]
        if idx.size == 1:
            meds[k] = idx[0]
        else:
            subD = D[np.ix_(idx, idx)]
            # 클러스터 내 거리합이 최소인 점이 메도이드
            local = np.argmin(subD.sum(axis=1))
            meds[k] = idx[local]
    return meds  # 0-based 데이터 인덱스

def per_client_medoid_dists(D: np.ndarray, labels: np.ndarray, *, index_base: int = 1) -> pd.DataFrame:
    """
    각 클라이언트 i에 대해:
      - self_medoid: 자신의 클러스터 메도이드 (표시용 index_base 적용)
      - nearest_other_medoid: 다른 클러스터 메도이드 중 가장 가까운 메도이드 (표시용)
      - d_self_medoid: 자신의 클러스터 메도이드까지 거리
      - d_min_other_medoid: 타 클러스터 메도이드까지의 최소 거리
      - margin_medoid: d_min_other_medoid - d_self_medoid (↑ 클수록 경계에서 안전)
    """
    n = labels.shape[0]
    # 각 클러스터 메도이드(0-based 데이터 인덱스)
    uniq = np.unique(labels)
    meds = np.empty(len(uniq), dtype=int)
    for k in uniq:
        idx = np.where(labels == k)[0]
        if idx.size == 1:
            meds[k] = idx[0]
        else:
            subD = D[np.ix_(idx, idx)]
            meds[k] = idx[np.argmin(subD.sum(axis=1))]

    d_self = np.zeros(n, dtype=float)
    d_other_min = np.full(n, np.nan, dtype=float)
    near_other_m = np.full(n, -1, dtype=int)  # 0-based 데이터 인덱스

    for i in range(n):
        k = int(labels[i])
        m_self = meds[k]
        d_self[i] = float(D[i, m_self])

        best_d = np.inf
        best_m = -1
        for kk in range(len(meds)):
            if kk == k:
                continue
            m_other = meds[kk]
            di = float(D[i, m_other])
            if di < best_d:
                best_d = di
                best_m = m_other
        if np.isfinite(best_d):
            d_other_min[i] = best_d
            near_other_m[i] = best_m

    idx = np.arange(n) + index_base
    # 표기용으로 index_base 적용 (near_other_m == -1이면 NaN로)
    nearest_other_disp = np.where(near_other_m >= 0, near_other_m + index_base, np.nan).astype(float)
    self_medoid_disp = meds[labels] + index_base

    df = pd.DataFrame({
        "client": idx,
        "cluster": labels,                         # 0..K-1
        "self_medoid": self_medoid_disp,           # 예: 16,17,21
        "nearest_other_medoid": nearest_other_disp,# 예: 21 등
        "d_self_medoid": d_self,
        "d_min_other_medoid": d_other_min,
    }).set_index("client").sort_index()

    # 마진 추가: (타 메도이드 최소거리 - 자기 메도이드 거리)
    df["margin_medoid"] = df["d_min_other_medoid"] - df["d_self_medoid"]

    return df




labels_c3 = labels_from_clusters_dict(c3_opt, n=53, one_based=True)  # 1-based 멤버 → 라벨(0..K-1)
df_ab    = per_client_ab_df(D, labels_c3, index_base=1)              # a(i), b(i), s(i)
df_md    = per_client_medoid_dists(D, labels_c3, index_base=1)       # 메도이드 기준 거리들

# 합치기 (client 인덱스로 조인)
df_all = df_ab.join(df_md[["self_medoid", "nearest_other_medoid", "d_self_medoid", "d_min_other_medoid", "margin_medoid"]])
print(df_all)

        cluster      a(i)      b(i)      s(i)  self_medoid  \
client                                                       
1             0  0.293510  0.362370  0.190028           16   
2             1  0.307963  0.440044  0.300154           17   
3             2  0.260763  0.378304  0.310705           21   
4             1  0.435658  0.502073  0.132282           17   
5             2  0.434462  0.443243  0.019813           21   
6             0  0.305694  0.246622 -0.193239           16   
7             1  0.270445  0.370914  0.270869           17   
8             1  0.344866  0.410143  0.159157           17   
9             1  0.273485  0.331327  0.174578           17   
10            0  0.311150  0.363231  0.143384           16   
11            1  0.297798  0.455869  0.346748           17   
12            1  0.270973  0.342247  0.208253           17   
13            0  0.339326  0.362455  0.063810           16   
14            1  0.307553  0.291416 -0.052468           17   
15      

In [10]:
import numpy as np
from typing import Dict, List

def add_medoid_and_reassign(
    D: np.ndarray,
    clusters_1: Dict[int, List[int]],     # 예: c3_opt (멤버 1-based)
    current_medoids_1: Dict[int, int],    # 예: {0:6, 1:7, 2:21}
    new_medoid_1: int = 20,               # 추가할 메도이드(1-based)
    eps: float = 1e-12,
) -> Dict[int, List[int]]:
    """
    기존 클러스터(멤버 1-based)와 메도이드(1-based)를 고정한 채,
    '새 메도이드(1-based)=20'을 추가하고, 각 클라이언트 i에 대해
    d(i,20) < min(d(i,6), d(i,7), d(i,21)) 이면 새 클러스터로 이동.
    결과는 동일 포맷(1-based 멤버)으로 반환.
    """
    n = D.shape[0]
    # 1) 0-based 변환
    old_meds0 = np.array([current_medoids_1[k] - 1 for k in sorted(current_medoids_1)], dtype=int)
    new_med0  = new_medoid_1 - 1

    # 라벨 배열(0..K-1) 구성: 현재 cluster dict에서 만들기
    K_old = len(clusters_1)
    labels = np.full(n, -1, dtype=int)
    for k, members in clusters_1.items():          # k는 0..K-1 라벨로 가정 (입력 그대로 유지)
        members0 = np.array(members, dtype=int) - 1
        labels[members0] = k
    assert (labels >= 0).all(), "라벨링되지 않은 인덱스가 있습니다."

    # 2) 각 포인트 i에 대해 '기존 메도이드들까지의 최소거리'와 '20까지의 거리' 비교
    d_to_old_meds_min = D[:, old_meds0].min(axis=1)   # (n,)
    d_to_new = D[:, new_med0]                         # (n,)

    # 3) 이동 조건: d(i,20) + eps < min(d(i,6), d(i,7), d(i,21))
    move_mask = d_to_new + eps < d_to_old_meds_min

    # 4) 새 라벨 부여: 새 클러스터 id = max 기존 라벨 + 1 (즉, 3)
    new_cluster_id = max(clusters_1.keys()) + 1
    labels_new = labels.copy()
    labels_new[move_mask] = new_cluster_id

    # 5) 결과 dict(1-based 멤버)로 재구성
    clusters_new_1 = {k: [] for k in clusters_1.keys()}
    clusters_new_1[new_cluster_id] = []
    for i0 in range(n):
        lab = int(labels_new[i0])
        clusters_new_1[lab].append(i0 + 1)  # 1-based로

    # 보기 좋게 멤버 정렬
    for k in clusters_new_1:
        clusters_new_1[k] = sorted(clusters_new_1[k])

    return clusters_new_1


In [11]:
# 현재 c3_opt과 메도이드 정보(1-based)
c3_opt = {
    0: [1, 6, 10, 13, 15, 16, 19, 20, 22, 26, 28, 29, 34, 39, 40, 43, 45, 46, 47, 48, 51],
    1: [2, 4, 7, 8, 9, 11, 12, 14, 17, 18, 23, 24, 25, 27, 30, 33, 35, 38, 42, 44, 49, 52, 53],
    2: [3, 5, 21, 31, 32, 36, 37, 41, 50],
}
medoids_1 = {0: 6, 1: 7, 2: 21}   # 각 클러스터의 메도이드(1-based)

# D는 0-based 거리행렬
c4_opt = add_medoid_and_reassign(D, c3_opt, medoids_1, new_medoid_1=20)

# 결과 확인
for k in sorted(c4_opt):
    print(f"cluster {k}: size={len(c4_opt[k])}, members={c4_opt[k]}")


cluster 0: size=6, members=[1, 6, 22, 29, 45, 48]
cluster 1: size=21, members=[4, 7, 9, 11, 12, 14, 17, 18, 23, 24, 25, 27, 30, 33, 35, 38, 42, 44, 49, 52, 53]
cluster 2: size=7, members=[3, 5, 21, 31, 32, 37, 41]
cluster 3: size=19, members=[2, 8, 10, 13, 15, 16, 19, 20, 26, 28, 34, 36, 39, 40, 43, 46, 47, 50, 51]


In [12]:
import numpy as np
import pandas as pd

def medoid20_margin(D: np.ndarray, medoid20: int = 20, old_medoids=(6,7,21), index_base: int = 1) -> pd.DataFrame:
    """
    모든 client i에 대해:
      d_i_20 = D[i, medoid20]             (20까지의 거리)
      d_i_oldmin = min(D[i, m] for m in {6,7,21})   (기존 메도이드들 중 최소거리)
      margin_20_vs_oldmin = d_i_20 - d_i_oldmin
    를 반환. (client index는 표시용으로 index_base 기준)
    """
    n = D.shape[0]
    m20_0 = medoid20 - index_base
    olds0 = np.array(old_medoids, dtype=int) - index_base

    d_i_20 = D[:, m20_0].astype(float)
    d_i_oldmin = D[:, olds0].min(axis=1).astype(float)
    margin = d_i_20 - d_i_oldmin

    # 어떤 기존 메도이드가 최소였는지도 같이 표시(1-based)
    nearest_old_arg = D[:, olds0].argmin(axis=1)
    nearest_old_medoid = (olds0[nearest_old_arg] + index_base).astype(int)

    df = pd.DataFrame({
        "client": np.arange(n) + index_base,
        "d_i_20": d_i_20,
        "d_i_oldmin": d_i_oldmin,
        "margin_20_vs_oldmin": margin,
        "nearest_old_medoid": nearest_old_medoid,  # 6/7/21 중 어느 것이 최소였는지
    }).set_index("client")

    return df


df_margin = medoid20_margin(D, medoid20=20, old_medoids=(6,7,21), index_base=1)
print(df_margin)                 # 모든 client 1..53의 값
# 더 가까운 애들만 보고 싶으면:
closer_to_20 = df_margin[df_margin["margin_20_vs_oldmin"] < 0]
print(closer_to_20)


          d_i_20    d_i_oldmin  margin_20_vs_oldmin  nearest_old_medoid
client                                                                 
1       0.339625  2.907142e-01             0.048911                   6
2       0.291993  3.188913e-01            -0.026899                   7
3       0.341501  2.025357e-01             0.138966                  21
4       0.511522  4.215428e-01             0.089979                   7
5       0.438348  4.381867e-01             0.000162                  21
6       0.277982  0.000000e+00             0.277982                   6
7       0.288860  0.000000e+00             0.288860                   7
8       0.253555  3.439761e-01            -0.090421                   7
9       0.280282  2.229521e-01             0.057330                   7
10      0.310221  3.445235e-01            -0.034302                   7
11      0.313706  3.119480e-01             0.001758                   7
12      0.275740  1.935102e-01             0.082230             